In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

#!conda install -c conda-forge bs4 --yes
from bs4 import BeautifulSoup

#!mamba install -c conda-forge html5lib --yes
# import html5lib

import requests # library to handle requests

print('Libraries imported.')

##### Scraping wiki url and create data frame

In [ ]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(wiki_url)
soup = BeautifulSoup(page.content, 'html.parser')

headings = ['Postal Code','Borough','Neighbourhood']


wiki_table = soup.find('table', class_= "wikitable sortable")

data = {'Postal Code':[], 'Borough':[],'Neighbourhood':[]}

rows = wiki_table.find_all('tr')

#print('number of rows: {}'.format(len(rows)))

row_marker = 0
for row in rows:
    column_marker = 0
    columns = row.find_all('td')
    for key, column in zip(data,columns):
        check = column.get_text()
        data[key].append(check[:-1])

df = pd.DataFrame(data)
df.head(11)


##### Dropping rows with Borough 'Not assigned'

In [ ]:
indexNames = df[df['Borough']== 'Not assigned'].index
indexNames
df_v2 = df.drop(indexNames).reset_index(drop=True)
df_v2.head(5)

##### For rows with **Not assigned** Neighbourhood, adopt Borough value for Neighbourhood

In [ ]:
indexNames2 = df_v2[df_v2['Neighbourhood']=='Not assigned'].index
df_v3 = df_v2

for ind in df_v2.index:
    if ind in indexNames2:
        df_v3.iloc[ind]['Neighbourhood']=df_v3.iloc[ind]['Borough']

df_v3.head(11)

##### Checking shape of final dataframe

In [ ]:
df_v3.shape

In [ ]:
# #!mamba install -c conda-forge geocoder --yes
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# for ind in df_v3.index:
#     postal_code = df_v3.iloc[ind]['Postal Code']
    
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
    
#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]
#     df_v3[ind]['Latitude'] = latitude
#     df_v3[ind]['Longitude'] = longitude

# df_v3.head(10)

##### Reading Geospatial_Coordinates.csv into a new dataframe

In [21]:
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')

##### Creating a new dataframe to include columns for geospatial coordinates

In [22]:
df_v4 = pd.merge(df_v3,df_coord, on = 'Postal Code', how='inner')
df_v4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
